In [24]:
# imports
import requests
import json
import csv
import time
import datetime
from string import Template
from Config import *
import sqlite3
from sqlite3 import Error
import matplotlib.pyplot as plt
import numpy as np

In [25]:
# Variables
headers = {"Authorization": API_TOKEN}
min_stars = 237
max_stars = 10000
last_activity = 90 # within the last __ days
created = 364 * 4 # within the last __ days
min_pull_num = 0 # amount of pull requests a repository needs
watchers = 0 # amount of watchers a repository needs

In [26]:
# Builds the query filter string compatible to github
def query_filter( min_stars, last_activity, created ):
    date_last_act = datetime.datetime.now() - datetime.timedelta( days=last_activity )
    date_created = datetime.datetime.now() - datetime.timedelta( days=created )
    stars = f'{min_stars}..1000'

    return f'is:public archived:false fork:false stars:{stars} pushed:20{date_last_act:%y-%m-%d}..* created:20{date_created:%y-%m-%d}..* sort:stars-asc'

In [27]:
# Funtion that uses requests.post to make the API call
def run_query(query, variables):
    request = requests.post('https://api.github.com/graphql', 
                            json={'query': query, 'variables' : variables}, headers=headers)
    
    #if request.status_code == 200:
    #if 'X-RateLimit-Reset' in request.headers :
    #    run_rate_lim_query()
        
    if 'Retry-After' in request.headers:  # reached retry limit
        print(f'[WAITING] for {request.headers["Retry-After"]} seconds before continuing...', end=' ')
        time.sleep(int(request.headers['Retry-After']))

    return request.json()
    #else:
    #    raise Exception(f'ERROR [{request.status_code}]: Query failed to execute...\nRESPONSE: {request.text}')

In [28]:
end_cursor = ""
owner = "astropy"
name = "astropy"
query_string = query_filter( min_stars, last_activity, created)
variables = {
    "owner" : owner,
    "name" : name,
    "end_cursor" : end_cursor,
    "pRsPerPage" : 1 # from 1 to 100
}

In [29]:
# setups the github graphql queries
def setup_query( variables, end_cursor ) :
    variables['end_cursor'] = end_cursor
    query = '''
    query($owner : String!, $name : String!, $pRsPerPage : Int) { 
      repository(owner: $owner, name: $name) {
        id
        name
        pullRequests (first: $pRsPerPage) {
          pageInfo {
            hasNextPage
            endCursor
          }
          nodes {
            author {
                login
            }
            id
            title
            number
            closed
            closedAt
            authorAssociation
            bodyText
            additions
            changedFiles
            deletions
            merged
            mergedAt
            state
            comments(first: 100) {
              edges {
                node {
                  id
                  author {
                    login
                  }
                  authorAssociation
                  bodyText
                  createdAt
                }
              }
            }
            reviewThreads(first:100) {
              edges {
                node {
                  comments(first:100){
                    nodes {
                      id
                      author {
                        login
                      }
                      authorAssociation
                      bodyText
                      createdAt
                    }
                  }
                }
              }
            }
          }
        }
    }
}'''
    query2 = '''
    query($owner : String!, $name : String!, $pRsPerPage : Int, $end_cursor : String!) { 
      repository(owner: $owner, name: $name) {
        id
        name
        pullRequests (first: $pRsPerPage, after: $end_cursor) {
          pageInfo {
            hasNextPage
            endCursor
          }
          nodes {
            author {
                login
            }
            id
            title
            number
            closed
            closedAt
            authorAssociation
            bodyText
            additions
            changedFiles
            deletions
            merged
            mergedAt
            state
            comments(first: 100) {
              edges {
                node {
                  id
                  author {
                    login
                  }
                  authorAssociation
                  bodyText
                  createdAt
                }
              }
            }
            reviewThreads(first:100) {
              edges {
                node {
                  comments(first:100){
                    nodes {
                      id
                      author {
                        login
                      }
                      authorAssociation
                      bodyText
                      createdAt
                    }
                  }
                }
              }
            }
          }
        }
    }
}'''

    if(end_cursor == "") : return (query, variables)
    else : return (query2, variables)

In [30]:
def setup_user_query() :
    query = '''
    query($username : String!) { 
  user(login: $username) {
    id
    pullRequests(first:1) {
      totalCount
    }
    repositories(first:1) {
      totalCount
    }
    repositoriesContributedTo(first:1) {
      totalCount
    }
  }
}'''
    return query

In [31]:
def setup_organization_query() :
    query = '''
    query($username : String!) { 
  organization(login: $username) {
    id
    repositories(first:1) {
      totalCount
    }
  }
}'''
    return query

In [32]:
# Runs the query and iterates through all pages of repositories
def iterate_queries( conn, create_dict_method ):

    print("[WORKING] Attemping to add repository ", variables["owner"], variables['name'], 
              "to the database.")
    
    end_cursor = ""
    end_cursor_string = ""
    hasNextPage = True
    index = 0
    
    ai = 1 # slow start: 1, 2, 4, 8 (max)
    md = 0.5
    
    print("[WORKING] Running script to collect all pullrequests. ")
    while( hasNextPage ):
        print("[WORKING] On page " + str(index))
        query = setup_query( variables, end_cursor )
        result = run_query( query[0], query[1] )
        #print(json.dumps(result))
        
        if 'errors' in result:
            if 'timeout' in result['errors'][0]['message']:  # reached timeout
                variables['pRsPerPage'] = int(max(1, variables['pRsPerPage'] * md))  # using AIMD
                ai = 1  # resetting slow start
                print('[WORKING] Timeout! - Reseting page size to : ' + 
                      str(variables['pRsPerPage']))
            if 'maximum' in result['errors'][0]['message']:  # reached timeout
                variables['pRsPerPage'] = int(max(1, variables['pRsPerPage'] * md))  # using AIMD
                ai = 1  # resetting slow start
                print('[WORKING] MAX_NODE_ERROR! - Reseting page size to : ' + 
                      str(variables['pRsPerPage']))
            else:  # some unexpected error.
                print(result['errors'])
                exit(1)
        
        if 'data' in result and result['data']:
            
            #print(json.dumps(result))
                      
            # insert_repositories( conn, result, create_dict_method )
            insert_users_prs_comments( conn, result )
            
            try:
                # if there is a next page, update the endcursor string and continue loop
                if( result["data"]["repository"]["pullRequests"]["pageInfo"]["hasNextPage"] ):
                    end_cursor = result["data"]["repository"]["pullRequests"]["pageInfo"]["endCursor"]
                    
                    variables['pRsPerPage'] = min(100, variables['pRsPerPage'] + ai)  # using AIMD
                    print("[WORKING] New pRsPerPage set to : " + str(variables['pRsPerPage']))
                    ai = min(8, ai * 2)  # slow start
                else:
                    if(result['data']['repository']['pullRequests']['totalCount'] > 1000) :
                        print('[WORKING] We reached the limit of 1,000 repositories.')
                        hasNextPage = False
                        end_cursor = ""
                    else :
                        hasNextPage = False
                        end_cursor = ""
                        print(json.dumps(result))
            except KeyError:
                print("[WORKING] No next page. ")
                break 

            index += 1
        time.sleep(1)

In [33]:
def create_user_tuple( conn, username ) :
    print("[WORKING] Running script to find " + username + " in github. ")
    user_variables = { 'username' : username }
    user_query = setup_user_query()
    result = run_query( user_query, user_variables )
    
    try:
        if( result['data']['user'] == None ) :
            orga_query = setup_organization_query()
            result = run_query( orga_query, user_variables )
            if( result['data']['organization'] == None ) :
                return (
                    username,
                    username,
                    None,
                    None,
                    None)
            else :
                return (
                    result['data']['organization']['id'],
                    username,
                    None,
                    result['data']['organization']['repositories']['totalCount'],
                    None)
        else :
            return (
                result['data']['user']['id'],
                username,
                result['data']['user']['pullRequests']['totalCount'],
                result['data']['user']['repositories']['totalCount'],
                result['data']['user']['repositoriesContributedTo']['totalCount'] )
    except: 
        print(json.dumps(result))
        return (
                    username,
                    username,
                    None,
                    None,
                    None)

In [34]:
def create_pr_tuple( node, repo_id, user_id ) :
    author = "ghost_user_does_not_exist"
    if(node['author']) : author = node['author']['login']
    
    return (
    node['id'],
    repo_id,
    user_id,
    author,
    node['number'],
    node['closed'],
    node['authorAssociation'],
    node['bodyText'],
    node['additions'],
    node['deletions'],
    node['changedFiles'],
    node['merged'],
    node['mergedAt'],
    node['state']
    )

In [35]:
def create_comment_tuple( node, pr_id, user_id ) :
    author = "ghost_user_does_not_exist"
    if(node['author']) : author = node['author']['login']
    
    return (
    node['id'],
    pr_id,
    user_id,
    author,
    node['authorAssociation'],
    node['bodyText'],
    node['createdAt']
    )

In [36]:
def find_user_id( username ) :
    sql  = "SELECT id FROM users WHERE users.name=\"{}\"".format( username )
    c = conn.cursor()
    c.execute(sql)
    rows = c.fetchone()
    if rows is not None:
        print("[SUCCESS] ", username, " is already added to the database. ")
        return rows[0]
    else: 
        user_tuple = create_user_tuple( conn, username )
        user_id = user_tuple[0] # get user id foriegn key
        return( user_tuple, user_id )

In [37]:
# insert repository tuples into the sqlite database
def insert_users_prs_comments( conn, result ):
    
    # initialize lists
    user_list = []
    pr_list = []
    comment_list = []
    
    # initialize sql queries
    user_sql = """INSERT or IGNORE INTO users (id, name, pr_total, repo_total, repo_contributed_total)
                                            VALUES (?, ?, ?, ?, ?)"""
    pr_sql = """INSERT or IGNORE INTO pull_requests (id, repo_id, user_id, author, number, closed,
                                                     authorAssoc, bodyText, additions, deletions,
                                                     changedFiles, merged, mergedAt, state) VALUES
                                                     (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    comment_sql = """INSERT or IGNORE INTO comments (id, pr_id, user_id, author, authorAssoc,
                                                  bodyText, createdAt) VALUES
                                                 (?, ?, ?, ?, ?, ?, ?)"""
    
    cur = conn.cursor()
    
    # repository id
    repo_id = result['data']['repository']['id']
    
    #print(json.dumps(result))
    
    # insert Pull Request
    pr_index = 0
    pr_nodes = result['data']['repository']['pullRequests']['nodes']
    for pr_node in pr_nodes:
        print("[WORKING] On pull request number: ", pr_index)
        
        if(pr_node['author']) :
            user_data = find_user_id( pr_node['author']['login'] )
            if isinstance( user_data, str ) : user_id = user_data
            else: 
                #f(user_data[0]) :
                    #ser_list.append( user_data[0] )
                print("[WORKING] Attemping to insert user into the database. ")
                cur.execute(user_sql, user_data[0])
                conn.commit()
                print("[SUCCESS] Inserted user into the database. ")
                user_id = user_data[1]
        else: user_id = None
        
        pr_tuple = create_pr_tuple( pr_node, repo_id, user_id )
        pr_id = pr_tuple[0] # get pr id foriegn key
        pr_list.append( pr_tuple )
        
        # insert Comments
        comm_edges = pr_node['comments']['edges']
        for comm_edge in comm_edges:
            if(comm_edge['node']['author']):
                user_data = find_user_id( comm_edge['node']['author']['login'] )
                if isinstance( user_data, str ) : user_id = user_data
                else: 
                    #f(user_data[0]) :
                        #ser_list.append( user_data[0] )
                    print("[WORKING] Attemping to insert user into the database. ")
                    cur.execute(user_sql, user_data[0])
                    conn.commit()
                    print("[SUCCESS] Inserted user into the database. ")
                    user_id = user_data[1] 
            else: user_id = None
            
            comment_tuple = create_comment_tuple( comm_edge['node'], user_id, pr_id )
            comment_list.append( comment_tuple )
        
        # insert Review Comments
        review_edges = pr_node['reviewThreads']['edges']
        for review_edge in review_edges:
            review_comm_nodes = review_edge['node']['comments']['nodes']
            for review_comm_node in review_comm_nodes:
                if(review_comm_node['author']) :
                    user_data = find_user_id( review_comm_node['author']['login'] )
                    if isinstance( user_data, str ) : user_id = user_data
                    else: 
                        #f(user_data[0]) :
                            #ser_list.append( user_data[0] )
                        print("[WORKING] Attemping to insert user into the database. ")
                        cur.execute(user_sql, user_data[0])
                        conn.commit()
                        print("[SUCCESS] Inserted user into the database. ")
                        user_id = user_data[1]
                else: user_id = None
                
                comment_tuple = create_comment_tuple( review_comm_node, user_id, pr_id )
                comment_list.append( comment_tuple )
        pr_index+=1
                

    
    #print("[WORKING] Attemping to insert users into the database. ")
    #cur.executemany(user_sql, user_list)
    #conn.commit()
    #print("[SUCCESS] Inserted users into the database. ")
    
    print("[WORKING] Attemping to insert pull requests into the database. ")
    cur.executemany(pr_sql, pr_list)
    conn.commit()
    print("[SUCCESS] Inserted pull requests into the database. ")
    
    print("[WORKING] Attemping to insert comments into the database. ")
    cur.executemany(comment_sql, comment_list)
    conn.commit()
    print("[SUCCESS] Inserted comments into the database. ")

    cur.close()
    return cur.lastrowid

In [38]:
def create_repo_tuple( node ) :
    commits = 0
    if(node["commits"]["target"]["history"]["totalCount"] != None) :
        commits = node["commits"]["target"]["history"]["totalCount"]
        
    primaryLanguage = "null"
    if(node["primaryLanguage"] != None) :
        primaryLanguage = node["primaryLanguage"]["name"]
        
    license_id = "0"
    license_name = "null"
    pseudoLicense = False
    if(node["licenseInfo"] != None) :
        license_id = node["licenseInfo"]["id"]
        license_name = node["licenseInfo"]["name"]
        pseudoLicense = node["licenseInfo"]["pseudoLicense"]
        
    return (
    node["id"], 
    node["name"], 
    node["owner"]["login"], 
    node["createdAt"], 
    node["isMirror"], 
    node["isFork"], 
    node["diskUsage"], 
    primaryLanguage,
    node["contributors"]["totalCount"], 
    node["watchers"]["totalCount"], 
    node["stargazers"]["totalCount"], 
    node["forkCount"], 
    node["issues"]["totalCount"], 
    commits, 
    node["pullRequests"]["totalCount"], 
    node["releases"]["totalCount"],
    license_id, 
    license_name, 
    pseudoLicense, 
    node["url"]
    )

In [39]:
def run_rate_lim_query():
    query = '''query { 
  rateLimit {
    resetAt
  }
}'''
    
    request = requests.post('https://api.github.com/graphql', 
                            json={'query': query}, headers=headers)
    
    ts = time.time()
    print(ts)
    print(request.headers['X-RateLimit-Reset'])
    sleep_time = float(request.headers['X-RateLimit-Reset']) - float(ts)
    #sleep_time = datetime.utcfromtimestamp(int(request.headers['X-RateLimit-Reset'])).strftime('%c')
    print(f'[WAITING] for {sleep_time} seconds before continuing...', end=' ')
    time.sleep(int(sleep_time))

    #else:
    #    raise Exception(f'ERROR [{request.status_code}]: Query failed to execute...\nRESPONSE: {request.text}')

In [40]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
    return conn

In [41]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [42]:
# insert repository tuples into the sqlite database
def insert_repositories( conn, result, create_dict_method ):
    
    data_list = []
    nodes = result["data"]["search"]["nodes"]
    for node in nodes:
        data_list.append( create_dict_method( node ) )
    
    """
    Create a new repository into the repository table
    :param conn:
    :param repository:
    :return: project id
    """
    sql = """INSERT or IGNORE INTO repositories (id, name, owner, start_date, isMirror, isFork, diskUsage, 
                     primaryLanguage, numContributors, watchers, stars, forks, issues, commits, 
                     pullRequests, releases, license_id, license_name, pseudoLicense, url) VALUES
                     (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
    
    cur = conn.cursor()
    cur.executemany(sql, data_list)
    conn.commit()
    print("[SUCCESS] Inserted repos into the database. ")
    cur.close()
    return cur.lastrowid

In [43]:
# name of the sqlite file
database = r".\databases\github_data.db"

# create a database connection
conn = create_connection(database)

In [44]:
# sql for creating the prs and comments tables
sql_create_prs_table = """CREATE TABLE IF NOT EXISTS pull_requests (
                                        id text PRIMARY KEY NOT NULL,
                                        repo_id text FORIEGN KEY NOT NULL,
                                        user_id text FORIEGN KEY,
                                        author text NOT NULL,
                                        number text NOT NULL,
                                        closed boolean NOT NULL,
                                        authorAssoc text NOT NULL,
                                        bodyText text NOT NULL,
                                        additions int NOT NULL,
                                        deletions int NOT NULL,
                                        changedFiles int NOT NULL,
                                        merged boolean NOT NULL,
                                        mergedAt text NOT NULL,
                                        state text NOT NULL
                                    ); """

sql_create_comments_table = """CREATE TABLE IF NOT EXISTS comments (
                                        id text PRIMARY KEY NOT NULL,
                                        pr_id text FORIEGN KEY NOT NULL,
                                        user_id text FORIEGN KEY,
                                        author text NOT NULL,
                                        authorAssoc text NOT NULL,
                                        bodyText text NOT NULL,
                                        createdAt text NOT NULL
                                    ); """

sql_create_users_table = """CREATE TABLE IF NOT EXISTS users (
                                        id text PRIMARY KEY,
                                        name text,
                                        pr_total int,
                                        repo_total int,
                                        repo_contributed_total int
                                    ); """

with conn:
    create_table(conn, sql_create_prs_table)
    create_table(conn, sql_create_comments_table)
    create_table(conn, sql_create_users_table)

In [45]:
sql_grab_repos = """SELECT id, owner, name, primaryLanguage FROM repositories 
                    WHERE primaryLanguage=\"JavaScript\" ORDER BY RANDOM() LIMIT 113;"""

c = conn.cursor()
result = c.execute(sql_grab_repos)
rows = c.fetchall()
rows = np.array(rows)

sql_create_working_table = """CREATE TABLE IF NOT EXISTS working_repos (
                                        id text PRIMARY KEY NOT NULL,
                                        owner text NOT NULL,
                                        name text NOT NULL,
                                        primaryLanguage text NOT NULL
                                    ); """
#with conn:
#    create_table(conn, sql_create_working_table)
#    sql = '''INSERT or IGNORE INTO working_repos (id, owner, name, primaryLanguage)
#            VALUES (?, ?, ?, ?)'''
#    cur = conn.cursor()
#    cur.executemany(sql, rows)
#    conn.commit()

In [46]:
sql_grab_repos = """SELECT id, owner, name, primaryLanguage FROM working_repos"""

c = conn.cursor()
result = c.execute(sql_grab_repos)
conn.commit()
rows = c.fetchall()
rows = np.array(rows)

size = rows.shape[0]

# when conn is valid
with conn:
    # run query to find all repositories
    # pass in the repository tuple builder
    # insert the tuple into the database
    index = 0
    for row in rows:
        global variables
        variables['owner'] = row[1]
        variables['name'] = row[2]
        #variables['owner'] = "potato4d"
        #variables['name'] = "nuxt-basic-auth-module"
        
        print("[----------------------------------------]")
        print("[UPDATE] On repostory ", index, "of ", size)
        print("[----------------------------------------]")
    
        iterate_queries( conn, create_repo_tuple )
        
        delete_sql = "DELETE from working_repos where id = ?"
        #print(delete_sql)
        c.execute(delete_sql, [row[0]])
        conn.commit()
        
        index+=1

[----------------------------------------]
[UPDATE] On repostory  0 of  6
[----------------------------------------]
[WORKING] Attemping to add repository  foundersandcoders master-reference to the database.
[WORKING] Running script to collect all pullrequests. 
[WORKING] On page 0
[{'type': 'RATE_LIMITED', 'message': 'API rate limit exceeded'}]
[WORKING] On page 0
[{'type': 'RATE_LIMITED', 'message': 'API rate limit exceeded'}]


KeyboardInterrupt: 